In [1]:
# Import dependiences
import pandas as pd
from sqlalchemy import create_engine
import re
import numpy as np

In [2]:
#Name variables for reading in CSVs
inspection_file = "./DOHMH_New_York_City_Restaurant_Inspection_Results.csv"
top_400_file = "./restaurant_week_2018_final.csv"

In [3]:
# Read in first csv to DF
inspection_orig_df = pd.read_csv(inspection_file)
inspection_orig_df.sample(10)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
314009,50057272,11 HANOVER GREEK,MANHATTAN,11,HANOVER SQ,10005.0,2127854000,Greek,02/21/2018,Violations were cited in the following area(s).,04H,"Raw, cooked or prepared food is adulterated, c...",Critical,74.0,NaN,NaN,05/04/2019,Cycle Inspection / Initial Inspection
33730,40726984,AKROTIRI,QUEENS,2920,30 AVENUE,11102.0,7187262447,Greek,02/20/2018,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,11.0,NaN,NaN,05/04/2019,Cycle Inspection / Initial Inspection
249610,50034079,THE RED DOORS BAR AND GRILL,BROOKLYN,1205,SURF AVE,11224.0,7188727177,American,09/18/2015,Violations were cited in the following area(s).,10I,"Single service item reused, improperly stored,...",Not Critical,30.0,NaN,NaN,05/04/2019,Pre-permit (Operational) / Initial Inspection
281564,50045701,SPORTS CAFE,BROOKLYN,172,AVENUE S,11223.0,7187433047,Eastern European,05/31/2016,Violations were cited in the following area(s).,10A,Toilet facility not maintained and provided wi...,Not Critical,14.0,NaN,NaN,05/04/2019,Pre-permit (Operational) / Initial Inspection
367994,50078920,TATOS PIZZA,BROOKLYN,8411,18TH AVE,11214.0,1718621330,Pizza,10/10/2018,Establishment re-opened by DOHMH,04L,Evidence of mice or live mice present in facil...,Critical,12.0,C,10/10/2018,05/04/2019,Pre-permit (Operational) / Reopening Inspection
66900,41150371,FAST & FRESH BURRITO DELI,BROOKLYN,84,HOYT STREET,11201.0,7188021661,Mexican,09/11/2017,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or condit...,Not Critical,9.0,A,09/11/2017,05/04/2019,Cycle Inspection / Re-inspection
275238,50044253,RUFFIAN WINE BAR,MANHATTAN,125,E 7TH ST,10009.0,2127770855,Continental,06/13/2016,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained;...,Not Critical,7.0,A,06/13/2016,05/04/2019,Pre-permit (Operational) / Initial Inspection
109801,41425803,DONUT SHOPPE,BROOKLYN,1503,AVENUE U,11229.0,7183752572,American,03/03/2017,Violations were cited in the following area(s).,04N,Filth flies or food/refuse/sewage-associated (...,Critical,21.0,B,03/03/2017,05/04/2019,Cycle Inspection / Re-inspection
167139,41676577,THE CUTTING ROOM,MANHATTAN,44,EAST 32 STREET,10016.0,2126911900,American,09/21/2018,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,12.0,A,09/21/2018,05/04/2019,Cycle Inspection / Re-inspection
308775,50055407,THE DEEP END,QUEENS,1080,WYCKOFF AVE,11385.0,6172302531,American,04/14/2018,Violations were cited in the following area(s).,06C,Food not protected from potential source of co...,Critical,9.0,A,04/14/2018,05/04/2019,Cycle Inspection / Re-inspection


In [19]:
# Read in second csv to DF
top_400_df = pd.read_csv(top_400_file)
top_400_df.head()

,name,street_address,google_map,review_count,phone,website,restaurant_type,average_review,food_review,service_review,...,star_1,star_2,star_3,star_4,star_5,description,restaurant_main_type,latitude,longitude,postal_code
0,Smoke Jazz and Supper Club,"2751 Broadway New York, NY 10025",//www.google.com/maps/search/?api=1&query=40.8...,2155,(212) 864-6662,http://www.smokejazz.com/,Contemporary American,4.421550,3.9,4.2,...,2,7,5,24,62,Smoke has augmented its reputation as one of M...,Contemporary American,40.801126,-73.968192,10025.0
1,Tavern on the Green,"1 Tavern on the Green New York, NY 10023",//www.google.com/maps/search/?api=1&query=40.7...,7029,(212) 877-8684,http://www.tavernonthegreen.com/,American,4.670290,4.4,4.4,...,1,2,6,19,72,Magical is a word thrown around a lot when dis...,American,40.772187,-73.977722,10023.0
2,ABC Kitchen,"35 East 18th Street New York, NY 10003",//www.google.com/maps/search/?api=1&query=40.7...,6031,(212) 475-5829,http://www.abckitchennyc.com/,Contemporary American,4.760310,4.6,4.4,...,1,2,5,15,77,ABC Kitchen with Jean-Georges: passionately co...,Contemporary American,40.737897,-73.989499,10003.0
3,Catch New York,"21 Ninth Avenue New York, NY 10014",//www.google.com/maps/search/?api=1&query=40.7...,4830,(212) 392-5978,http://www.catchhg.com/,Seafood,4.548300,4.6,4.2,...,3,2,6,15,74,CATCH features contemporary renditions of clas...,Seafood,40.740227,-74.006340,10014.0
4,Becco,"355 West 46th Street New York, NY 10010",//www.google.com/maps/search/?api=1&query=40.7...,18139,(212) 397-7597,http://www.becco-nyc.com/,Italian,4.418139,4.4,4.4,...,1,3,9,27,60,Lidia Bastianich & her son Joe's Theater Distr...,Italian,40.760804,-73.989832,10010.0


In [20]:
#clean the DF's
clean_inspection_df = inspection_orig_df[["CAMIS", "DBA", "GRADE", "VIOLATION DESCRIPTION", "GRADE DATE", "ZIPCODE"]].copy()
clean_top_400_df = top_400_df[["name", "average_review"]].copy()


In [21]:
# clean up the column names
inspection_transformed = clean_inspection_df.rename(columns={"CAMIS":"id",
                                                            "DBA":"name",
                                                            "VIOLATION DESCRIPTION":"violation",
                                                            "GRADE":"grade",
                                                            "GRADE DATE":"grade_date",
                                                            "ZIPCODE": "zipcode"})

#Remove decimal in zipcode column
inspection_transformed["zipcode"] = inspection_transformed["zipcode"].astype(str)
inspection_transformed["zipcode"]= inspection_transformed["zipcode"].str.replace('.0', ' ')
inspection_transformed.head()

,id,name,grade,violation,grade_date,zipcode
0,30075445,MORRIS PARK BAKE SHOP,A,Pesticide use not in accordance with label or ...,05/11/2018,462
1,30075445,MORRIS PARK BAKE SHOP,A,Non-food contact surface improperly constructe...,05/11/2018,462
2,30075445,MORRIS PARK BAKE SHOP,A,"Food contact surface not properly washed, rins...",05/18/2017,462
3,30075445,MORRIS PARK BAKE SHOP,A,Non-food contact surface improperly constructe...,05/18/2017,462
4,30075445,MORRIS PARK BAKE SHOP,A,Evidence of mice or live mice present in facil...,02/18/2016,462


In [22]:
#Remove special characters
inspection_transformed['violation'] = inspection_transformed["violation"].replace(r'[^a-zA-Z0-9 ]\s?',r'',regex=True)
clean_top_400_df["name"] = clean_top_400_df["name"].replace(r'[^a-zA-Z0-9 ]\s?',r'',regex=True)

#Remove empty values
inspection_table = inspection_transformed.dropna(axis=0)
rating_table = clean_top_400_df.dropna(axis=0)

In [24]:
#connect to local DB
rds_connection_string = "root:rjdGr0n@@127.0.0.1/project_db"
engine = create_engine(f'mysql://{rds_connection_string}', encoding='utf-8')

In [27]:
# Check for tables
engine.table_names()

['inspection_table', 'rating_table']

In [26]:
# finally load the df's into DB
inspection_table.to_sql(name='inspection_table', con=engine, if_exists='append', index=True)
rating_table.to_sql(name='rating_table', con=engine, if_exists='append', index=True)